In [61]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, BayesianRidge, Ridge
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.experimental import enable_iterative_imputer  # required
from sklearn.impute import SimpleImputer, IterativeImputer
from sklearn.ensemble import StackingRegressor
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd

In [47]:
dataset = []
dataset.append(pd.read_csv("http://nucsmb/ensemble/income_small0.csv"))
dataset.append(pd.read_csv("http://nucsmb/ensemble/income_small1.csv"))
dataset.append(pd.read_csv("http://nucsmb/ensemble/income_small2.csv"))
dataset.append(pd.read_csv("http://nucsmb/ensemble/income_small3.csv"))

In [48]:
datasets0 = pd.concat(dataset)
datasets = datasets0.dropna(subset=['Income']) #drop rows with missing target variable
X = datasets.drop('Income', axis=1)       #obtain features
y = datasets['Income']                    #obtain target variable
ALL_FEATURES = X.columns.tolist()                      #store feature names
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42) #0.3 because dataset is small

In [49]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 34 entries, 0 to 9
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Name              24 non-null     object 
 1   City              24 non-null     object 
 2   State             24 non-null     object 
 3   Occupation        24 non-null     object 
 4   Age               20 non-null     float64
 5   Years_Experience  10 non-null     float64
 6   Education_Years   10 non-null     float64
 7   Hours_Per_Week    10 non-null     float64
 8   Commute_Distance  10 non-null     float64
dtypes: float64(5), object(4)
memory usage: 2.7+ KB


In [50]:
X.isnull().sum()  # Shows count of nulls in each feature column. Will be handled by imputation

Name                10
City                10
State               10
Occupation          10
Age                 14
Years_Experience    24
Education_Years     24
Hours_Per_Week      24
Commute_Distance    24
dtype: int64

In [51]:
transformer = ColumnTransformer(transformers=[
    ('name', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))  # Changed 'scaler' to 'encoder'
    ]), ['Name']),
    ('city', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ]), ['City']),
    ('state', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ]), ['State']),   
    ('occupation', Pipeline([
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('encoder', OneHotEncoder(handle_unknown='ignore'))
    ]), ['Occupation']),
    ('age', Pipeline([
        ('imputer', IterativeImputer(estimator=BayesianRidge(), random_state=42)),   
        ('scaler', StandardScaler()) 
    ]), ['Age']),
    ('experience', Pipeline([
        ('imputer', IterativeImputer(estimator=BayesianRidge(), random_state=42)),  
        ('scaler', StandardScaler()) 
    ]), ['Years_Experience']),   
    ('education', Pipeline([
        ('imputer', IterativeImputer(estimator=BayesianRidge(), random_state=42)), 
        ('scaler', StandardScaler()) 
    ]), ['Education_Years']),
    ('hours', Pipeline([
        ('imputer', IterativeImputer(estimator=BayesianRidge(), random_state=42)),  
        ('scaler', StandardScaler()) 
    ]), ['Hours_Per_Week']),   
    ('commute', Pipeline([
        ('imputer', IterativeImputer(estimator=BayesianRidge(), random_state=42)),  
        ('scaler', StandardScaler())  
    ]), ['Commute_Distance'])
], remainder='drop')

In [52]:
linear_pipeline = Pipeline(steps=[
    ('transformer', transformer), 
    ('model', LinearRegression())
])

In [53]:
random_forest_pipeline = Pipeline(steps=[
    ('transformer', transformer), 
    ('model', RandomForestRegressor(random_state=42))
])

In [54]:
gradient_boosting_pipeline = Pipeline(steps=[
    ('transformer', transformer), 
    ('model', GradientBoostingRegressor(random_state=42))
])

In [55]:
decision_tree_pipeline = Pipeline(steps=[
    ('transformer', transformer), 
    ('model', DecisionTreeRegressor(max_depth=5, random_state=42))
])

In [56]:
estimators = [
            ('linear_model', linear_pipeline),
            ('random_forest_model', random_forest_pipeline),
            ('gradient_boosting_model', gradient_boosting_pipeline),
            ('decision_tree_model', decision_tree_pipeline)
        ]

stacking_model = StackingRegressor(
    estimators=estimators,
    final_estimator=Ridge(alpha=1.0),
    cv=5  # 5-fold cross-validation (default)
)

In [57]:
stacking_model.fit(X_train, y_train)

,estimators,"[('linear_model', ...), ('random_forest_model', ...), ...]"
,final_estimator,Ridge()
,cv,5
,n_jobs,None
,passthrough,False
,verbose,0
,transformers,"[('name', ...), ('city', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None


In [59]:
#fit base models also
linear_pipeline.fit(X_train, y_train)
random_forest_pipeline.fit(X_train, y_train)
gradient_boosting_pipeline.fit(X_train, y_train)
decision_tree_pipeline.fit(X_train, y_train)

,steps,"[('transformer', ...), ('model', ...)]"
,transform_input,None
,memory,None
,verbose,False
,transformers,"[('name', ...), ('city', ...), ...]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True


In [62]:
# Create a dictionary of our fitted models
models = {
    "Stacking Ensemble": stacking_model,
    "Linear Regression": linear_pipeline,
    "Random Forest": random_forest_pipeline,
    "Gradient Boosting": gradient_boosting_pipeline,
    "Decision Tree": decision_tree_pipeline
}

# Iterate through each model, make predictions on the test set, and calculate metrics
for name, model in models.items():
    # Make predictions on the test data
    y_pred = model.predict(X_test)
    
    # Calculate metrics
    r2 = r2_score(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    
    # Print metrics
    print(f"\nModel: {name}")
    print(f"  R-squared (R²): {r2:.4f}")
    print(f"  Mean Absolute Error (MAE): ${mae:,.2f}")
    print(f"  Root Mean Squared Error (RMSE): ${rmse:,.2f}")

print("\n--- Validation Complete ---")


Model: Stacking Ensemble
  R-squared (R²): -0.0458
  Mean Absolute Error (MAE): $4,417.15
  Root Mean Squared Error (RMSE): $5,968.36

Model: Linear Regression
  R-squared (R²): -0.1160
  Mean Absolute Error (MAE): $3,984.04
  Root Mean Squared Error (RMSE): $6,165.50

Model: Random Forest
  R-squared (R²): -0.0565
  Mean Absolute Error (MAE): $4,464.27
  Root Mean Squared Error (RMSE): $5,998.93

Model: Gradient Boosting
  R-squared (R²): -0.0235
  Mean Absolute Error (MAE): $4,456.44
  Root Mean Squared Error (RMSE): $5,904.33

Model: Decision Tree
  R-squared (R²): -1.3727
  Mean Absolute Error (MAE): $7,414.14
  Root Mean Squared Error (RMSE): $8,990.02

--- Validation Complete ---


In [58]:
# Example 1: Single prediction with all features
new_data = pd.DataFrame({
    'Name': ['Sarah Johnson'],
    'City': ['Boston'],
    'State': ['MA'],
    'Occupation': ['Data Scientist'],
    'Age': [35],
    'Years_Experience': [10],
    'Education_Years': [18],
    'Hours_Per_Week': [45],
    'Commute_Distance': [8.5]
})

# Make prediction
predicted_income = stacking_model.predict(new_data)
print(f"Predicted Income: ${predicted_income[0]:,.2f}")


# Example 2: Multiple predictions at once
new_data_batch = pd.DataFrame({
    'Name': ['John Smith', 'Maria Garcia', 'David Lee'],
    'City': ['Seattle', 'Austin', 'Denver'],
    'State': ['WA', 'TX', 'CO'],
    'Occupation': ['Engineer', 'Manager', 'Designer'],
    'Age': [28, 42, 31],
    'Years_Experience': [5, 18, 8],
    'Education_Years': [16, 20, 14],
    'Hours_Per_Week': [50, 45, 40],
    'Commute_Distance': [12.3, 6.7, 15.2]
})

# Make batch predictions
predicted_incomes = stacking_model.predict(new_data_batch)
for i, income in enumerate(predicted_incomes):
    print(f"Person {i+1} Predicted Income: ${income:,.2f}")


# Example 3: Prediction with missing values (handled by imputers)
new_data_missing = pd.DataFrame({
    'Name': ['Emily Chen'],
    'City': ['Portland'],
    'State': ['OR'],
    'Occupation': ['Analyst'],
    'Age': [None],  # Missing value - will be imputed
    'Years_Experience': [7],
    'Education_Years': [16],
    'Hours_Per_Week': [None],  # Missing value - will be imputed
    'Commute_Distance': [10.5]
})

predicted_income = stacking_model.predict(new_data_missing)
print(f"Predicted Income (with missing values): ${predicted_income[0]:,.2f}")


# Example 4: Get predictions from individual base models
# (useful for understanding how each model contributes)
linear_pred = linear_pipeline.predict(new_data)
rf_pred = random_forest_pipeline.predict(new_data)
gb_pred = gradient_boosting_pipeline.predict(new_data)
dt_pred = decision_tree_pipeline.predict(new_data)

print(f"\nBase Model Predictions:")
print(f"Linear Regression: ${linear_pred[0]:,.2f}")
print(f"Random Forest: ${rf_pred[0]:,.2f}")
print(f"Gradient Boosting: ${gb_pred[0]:,.2f}")
print(f"Decision Tree: ${dt_pred[0]:,.2f}")
print(f"Stacking Ensemble: ${predicted_income[0]:,.2f}")

Predicted Income: $74,938.36
Person 1 Predicted Income: $84,380.43
Person 2 Predicted Income: $74,182.85
Person 3 Predicted Income: $63,571.28
Predicted Income (with missing values): $71,117.65


NotFittedError: Pipeline is not fitted yet.